In [7]:
pip install torch pymupdf torch protobuf sentencepiece accelerate

  Using cached pymupdf-1.25.5-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 108.6 MB/s eta 0:00:00
  Attempting uni

In [8]:
import fitz  # PyMuPDF
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login

In [9]:
login("")

In [10]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")


Device set to use cuda:0


In [12]:
def extract_data(path: str) -> list[str]:
    doc = fitz.open(path)
    return [page.get_text() for page in doc]

In [14]:
# Load and parse PDF
pages = extract_data("/content/sterling-accuris-pathology-sample-report-unlocked.pdf")

# Define output dictionary
all_structured_data = {}

In [15]:
for i, page in enumerate(pages):
    prompt = f"""
You are a medical-report parser. I will provide you with raw text from one page of a pathology report.

Your task is to extract and return a single valid JSON object with the following structure:

1. Patient Information:
    - patient_name: string
    - hospital_name: string
    - doctor_names: list of strings
    - sex: string
    - lab_id: string
    - age: integer
    - dob: string (YYYY-MM-DD or null)

2. Sections of the report (e.g., "Complete Blood Count"):
    - Each section’s value must be a list of test records with:
        - name: string
        - method: string
        - value: number
        - unit: string
        - ref_low: number or null
        - ref_high: number or null

Only emit a valid JSON object. No markdown or extra commentary.

Here is the raw page text:
{page}
"""

    try:
        result = generator(prompt, max_new_tokens=1024, do_sample=False)
        output_text = result[0]['generated_text']

        # Extract the JSON part if there's extra text
        start = output_text.find('{')
        end = output_text.rfind('}') + 1
        json_text = output_text[start:end]

        structured_data = json.loads(json_text)
        all_structured_data[f"Page {i+1}"] = structured_data
        print(f"Page {i+1} processed successfully.")

    except Exception as e:
        print(f"Error processing Page {i+1}: {e}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error processing Page 1: Expecting ',' delimiter: line 79 column 17 (char 1619)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Page 2 processed successfully.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Page 16 processed successfully.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Page 17 processed successfully.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error processing Page 18: Expecting ',' delimiter: line 124 column 10 (char 2884)
Error processing Page 19: Expecting ',' delimiter: line 121 column 6 (char 2522)


In [16]:
with open("structured_medical_data.json", "w") as f:
    json.dump(all_structured_data, f, indent=4)

In [21]:
json_text

'{\n  "patient_name": "Lyubochka Svetka",\n  "hospital_name": "Sterling Accuris Buddy",\n  "doctor_names": ["Dr. Sanjeev Shah", "Dr.Yash Shah", "DR.TEJASWINI DHOTE"],\n  "sex": "Male",\n  "lab_id": "02232160XXXX",\n  "age": 41,\n  "dob": "1982-02-01",\n  "sections": [\n    {\n      "name": "Hemoglobin",\n      "method": "Colorimetric",\n      "value": 14.5,\n      "unit": "g/dL",\n      "ref_low": 13.0,\n      "ref_high": 16.5\n    },\n    {\n      "name": "RBC Count",\n      "method": "Electrical impedance",\n      "value": 4.79,\n      "unit": "million/cmm",\n      "ref_low": 4.5,\n      "ref_high": 5.5\n    },\n    {\n      "name": "Hematocrit",\n      "method": "Calculated",\n      "value": 43.3,\n      "unit": "%"\n    },\n    {\n      "name": "MCV",\n      "method": "Derived",\n      "value": 90.3,\n      "unit": "fL",\n      "ref_low": 83,\n      "ref_high": 101\n    },\n    {\n      "name": "MCH",\n      "method": "Calculated",\n      "value": 30.2,\n      "unit": "pg"\n    },\